In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import NMF

In [8]:
import glob
import os
from collections import defaultdict
import re
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [9]:
import nltk
import os
import spacy
import re
import string
import pickle
from itertools import cycle
import matplotlib.pyplot as plt
from nltk import stem
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']
from nltk.tokenize import word_tokenize, sent_tokenize, MWETokenizer
from gensim import corpora, models, similarities, matutils
from sklearn.decomposition import TruncatedSVD, NMF
from corextopic import corextopic, vis_topic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import scattertext as st

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


### Stop Words

In [13]:
nonwords=['Skip to content', 'All Results', 'Home', 'News', 'Reviews', 'Albums', 'Tracks', 'Sunday Reviews', '8.0+ Reviews',
          'Best New Music', 'Features', 'Lists & Guides', 'Longform', 'Rising', 'Photo Galleries', 'The Pitch', 'Video',
          'Over/Under', 'Liner Notes', 'Under the Influences', 'On the Records', 'Critical Breakthroughs', 'Podcast', 'GRAMMYS 2021',
          'Newsletter', 'Advertising', 'Masthead', 'Careers', 'Contact', 'Accessibility Help', 'More Pitchfork', 'Pitchfork Music Festival Chicago',
          'Pitchfork Music Festival Paris', 'Pitchfork Music Festival Berlin', 'Pitchfork Radio', 'Home     ','GRAMMYS', 
          'Staff Picks', 'Bandcamp', 'Buy','8.0+ ','8.0+','Back to home', 'Sign','newsletter','here','pitchfork','Pitchfork']


In [14]:
from sklearn.feature_extraction import text 

my_stop_words = nltk.corpus.stopwords.words('english')
my_stop_words.extend(nonwords)

### Lemmatize
**Lemmatizing could be tricky since many common words can be used within artist names (tribe called quest becomes tribe call**

In [15]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
spacy_lemmatize = lambda x: " ".join([token.lemma_ for token in nlp(x)])

In [16]:
norah="The restive jazz star, who went on to work with Jack White, Danger Mouse, and others after¬†her multiplatinum debut, returns with a no-frills record that fades into the background without much fuss. At this point of her career, it‚Äôs easy to forget that Norah Jones was once considered a jazz singer. Back in 2002, long before she worked with Jack White and Brian ‚ÄúDanger Mouse‚Äù Burton, Jones released Come Away With Me, an understated gem that went on to sell over 11 million units and earn eight Grammy awards‚including Album of the Year and Song of the Year for ‚Don't Know Why. ‚Armed with that voice‚wry, simmering inflection‚the Texas native has proven she can sing anything, and sound natural doing so, no matter where the road has taken her. That said, its been tough to assess Jones creative arc. Feels Like Home, her 2004 sophomore LP, eschewed the traditional jazz of the first record for a folk/country sound. She's released two honky-tonk country albums as a member of the Little Willies; and in 2012, Jones dropped Little Broken Hearts, a concept record about emotional anguish, set against Danger Mouse‚ soulful pop arrangements. Elsewhere, Jones had notable features on Mouse and composer Daniele Luppi‚Rome; with Q-Tip, OutKast and Talib Kweli on their respective albums; and on the Robert Glasper Experiment  Black Radio 2. Throughout all this, Jones has remained somewhat anonymous, mainly because she‚Äôs never appealed to pop and hip-hop consumers. Her music speaks to the adult contemporary crowd, those who actually buy CDs instead of streaming online. Jones has maintained a nice career largely out of the public eye; now 14 years removed from her seminal work, the musician returns to initial form on Day Breaks, her sixth studio album. In a recent interview with the New York Times, Jones said the genesis of the LP can be traced back to a 2014 performance at the Kennedy Center in Washington, D.C., during a 75th anniversary concert for her label, Blue Note Records. She was performing a cover of songwriter Jesse Harris‚Äô ‚ÄúI‚Äôve Got to See You Again‚Äù with iconic saxophonist Wayne Shorter, drummer Brian Blade, and bassist John Patitucci. ‚ÄúWhen I started thinking about making a ‚Äòjazz record,‚Äô mostly I was thinking about recording with Wayne and Brian,‚Äù Jones told the Times. ‚ÄúI didn‚Äôt want it to be standards. I was hoping for something very rhythmic, with Wayne floating over the top.‚Äù Shorter, Blade, and organist Dr. Lonnie Smith appear throughout Day Breaks, an efficient 12-track set featuring nine original songs and three covers: Horace Silver‚Äôs ‚ÄúPeace,‚Äù Duke Ellington‚Äôs ‚ÄúFleurette Africaine‚Äù and Neil Young‚Äôs ‚ÄúDon‚Äôt Be Denied.‚Äù Day Breaks is especially sparse, a no-frills record that fades into the background without much fuss. It seems to reflect the singer‚Äôs personal and professional comfort, that‚Äîafter 15 years as a signed artist with more than 50 million records sold‚ÄîJones doesn‚Äôt need to adhere to industry pressures to remain relevant. Whereas some artists revert to their best-received work as a way to reignite past glory, Day Breaks feels like the logical next step for a singer who‚Äôs done just about everything there is to do musically. This one isn't a barn-burner, but it's not supposed to be. Unfortunately, though, Day Breaks grows a bit tedious near the middle, and it's easy to forget it's playing if you aren't paying attention. Lyrically, Day Breaks¬†embraces¬†a hazy‚Äîif not melancholy‚Äîvibe, similar to the Billie Holiday albums Jones listened to as a child. These tracks address some level of perseverance, a pushing through to better days whether romantically or socially. Jones uses a conversational cadence to tell these stories, bolstering the narrative and giving her words better impact. On ‚ÄúFlipside,‚Äù a song about racial and civic injustice, Jones and the band sound especially poignant. The other songs slow to a crawl, but this one is hard-charging with a strong message. ‚ÄúIf we‚Äôre all free, then why does it seem we can‚Äôt just be?‚Äù Jones asks. ‚ÄúPut the guns away or we‚Äôre all gonna lose.‚Äù (As of this review, two more unarmed black men‚ÄîTerence Crutcher in Tulsa, Oklahoma; Keith Lamont Scott in Charlotte, North Carolina‚Äîwere shot and killed by police for no reason.) And while Jones didn‚Äôt write the lyrics to ‚ÄúPeace‚Äù‚Äîit is a cover, after all‚Äîshe effectively owns the message. ‚Peace,she hums, is for everyone.‚Äù Sure that‚Äôs a noble idea, but w've got a long way to go."
charles="best new reissue At last available digitally, these quietly revolutionary albums still break one of music‚Äôs deepest-held genre orthodoxies. More than a half-century later, the premise of Ray Charles‚Äô two-volume Modern Sounds in Country and Western Music‚Äîthat is, an established R&B star makes a record of country hits‚Äîmight yet inspire a spit take from the uninitiated. Alas, the uninitiated have long remained uninitiated, since these groundbreaking albums have been reissued infrequently since their 1962 release. But a remastered Concord set, which includes vinyl and CD reprints of both albums, marks the first time these landmarks are available as downloads and streams. Maybe having them available outside of distorted YouTube rips will, 57 years later, affirm Charles‚Äô rightful status as a country music pioneer. Building off a string of hits with Atlantic and an ABC-Paramount contract with a ‚Äúfull artistic freedom‚clause, Charles was finally able to record the music he‚Äôd been listening to since he was a self-described country boy growing up in Greenville, FL., a town of around 900 near the Georgia border. Back then, he would tune into the Grand Ole Opry every Saturday night. That period provided direct inspiration for the first Modern Sounds and its hastily assembled sequel, recorded in the wake of its predecessor‚ explosive success. Charles put his golden touch on two dozen of the biggest country songs of the day. ‚ÄúI think a lot of the hillbilly music is wonderful,‚Äù he told Billboard, starting his campaign to record such tunes in 1959. ‚ÄúI think I could do a good job with the right hillbilly song today. If you really have the ability, that‚Äôs what counts.‚Äù Despite several subsequent counterexamples, from Charley Pride to Darius Rucker, the idea that a black singer could attain mainstream success singing country music still challenges one of the deepest-held genre orthodoxies. ‚ÄúA rhythm-and-blues singer, at least when I was coming up, was typecast, much like an actor,‚Äù Charles wrote in his 1978 autobiography. ‚ÄúEven today I‚Äôm called ‚ÄòRay Charles, the blues singer,‚Äô or ‚ÄòRay Charles, the soul singer.‚Äô And this is after 20 years of singing damn near everything!‚Äù Charles approached the album with what would turn out to be necessary defiance, as he faced skepticism from his label and the industry at large. Even though the albums contain some of Charles‚Äô most enduring singles, they are still more accepted as Ray Charles songs than disclaimer-free country. When Modern Sounds was first released, the charts refused to track this music as country; the Grammys nominated these very clearly named albums in rhythm & blues categories. Still, these arrangements played off the perceived tension of Charles‚Äô foray into Nashville territory. Half the songs were recorded in Los Angeles, with Charles backed by lush strings and a straight-laced choir to build a poppier take on the pervasive Nashville Sound. In New York, a swinging big band more familiar to those who were already Charles fans supported the other half. The end result is seamless, showing just how wide a gap Charles‚Äô voice could bridge. String-backed ballad ‚ÄúI Can‚Äôt Stop Loving You‚Äù is a larger-than-life song that toes the line of pure schlock with lyrical hyperbole and musical gloss. But a gentle, insistent country shuffle and Charles‚Äô endless capacity for relatable sorrow (perhaps country music‚Äôs most enduring hallmark) ground it. Buried in the album‚Äôs B-side because producer Sid Feller thought it was the weakest track, the song became the biggest hit of Charles‚Äô career. Charles‚Äô mix of polish and pure emotion has proven nearly impossible to match, from weepy orchestral tunes like ‚ÄúYou Don‚Äôt Know Me‚Äù (maybe the most convincing description of life in the friend zone ever recorded) to brassy up-tempo numbers, like Hank Williams‚Äô ‚ÄúHey, Good Lookin‚Äô.‚Äù But making something infallibly bright and genuine, rich and timeless was a secondary achievement. More importantly, he proved that the connection he heard between country‚Äôs twang and R&B‚Äôs groove wasn‚Äôt a fatal distinction but simply a matter of interpretation. Charles exposed how inane it is to draw a hard line between two genres that shared an ancestor‚Äîthe United States‚Äô original hybrid, the blues. You can go through and parse which parts of every record sound more like Nashville, which parts sound more like soul. ‚ÄúYou Are My Sunshine‚Äù relies on the same hip-swiveling call-and-response that Motown would soon make ubiquitous. ‚ÄúBorn to Lose‚Äù uses barely any swing outside of Charles‚Äô voice, readying it for the honky-tonk. But that kind of dissection misses the point‚Äîthat this music is accessible to him, too, a black man. In the end, Charles didn‚Äôt just fit in; he revolutionized the genre by sparking a rush of Nashville/pop crossover acts. This music remains a tribute to and rejoinder of the futile divisions we so often take for granted. Charles made headlines promoting the album with integrated shows throughout the South, refusing to perform in segregated venues. ‚ÄúBoth whites and Negroes alike have attended the same dance and actually danced together at Charles concerts,‚Äù wrote the Chicago Daily Defender in November 1962 of Charles‚Äô shows in Memphis and St. Louis. ‚ÄúThere is no race problem when he plays.‚Äù It‚Äôs proof of Charles‚Äô enduring yet still-aspirational achievement on these albums: unity"

In [31]:
df = pd.DataFrame([norah], columns=['review'])
df

,review
0,"The restive jazz star, who went on to work wit..."


In [41]:
df2 = pd.DataFrame([charles], columns=['review'])
df3=df.append(df2)

In [42]:
df3["review_lem"] = df3["review"].apply(spacy_lemmatize)
df3.head(2)

,review,review_lem
0,"The restive jazz star, who went on to work wit...","the restive jazz star , who go on to work with..."
0,"best new reissue At last available digitally, ...","good new reissue at last available digitally ,..."


In [34]:
breaks = lambda x: re.sub("\n", " ", x)
alphanumeric = lambda x: re.sub("\w*\d\w*", "", x)
punc_lower = lambda x: re.sub("[%s]" % re.escape(string.punctuation), "", x.lower())
#punc_lower = lambda x: re.sub("[%s]" % re.escape(string.punctuation), "",x ) # x.lower())

In [43]:
df3["review_clean"] = df3["review"].map(breaks).map(alphanumeric).map(punc_lower)


In [44]:
df3.head(2)

,review,review_lem,review_clean
0,"The restive jazz star, who went on to work wit...","the restive jazz star , who go on to work with...",the restive jazz star who went on to work with...
0,"best new reissue At last available digitally, ...","good new reissue at last available digitally ,...",best new reissue at last available digitally t...


### Apply matrix to REVIEW CLEAN text - this is what's used for COSINE SIM

In [46]:
tfidf = TfidfVectorizer(analyzer='word', stop_words=my_stop_words)

In [47]:
rev_matrix= tfidf.fit_transform(df3['review_clean'])

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['2021', 'accessibility', 'advertising', 'albums', 'back', 'bandcamp', 'berlin', 'best', 'breakthroughs', 'buy', 'careers', 'chicago', 'contact', 'content', 'critical', 'features', 'festival', 'galleries', 'grammys', 'guides', 'help', 'home', 'influences', 'liner', 'lists', 'longform', 'masthead', 'music', 'new', 'news', 'notes', 'paris', 'photo', 'picks', 'pitch', 'podcast', 'radio', 'records', 'results', 'reviews', 'rising', 'sign', 'skip', 'staff', 'sunday', 'tracks', 'video'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [48]:
rev_matrix.shape

(2, 707)

In [49]:
rev_matrix

<2x707 sparse matrix of type '<class 'numpy.float64'>'
	with 762 stored elements in Compressed Sparse Row format>

In [50]:
# take a peek at the each review and words

print('Sparse Matrix')
# A compressed version; the "sparse" matrix.
print(type(rev_matrix))
#print(rev_matrix)[0][0]

Sparse Matrix
<class 'scipy.sparse.csr.csr_matrix'>


In [51]:
# change to array for dataframe 

print ('Matrix')
rev_matrix_arr = rev_matrix.toarray()
print(type(rev_matrix_arr))
print(rev_matrix_arr)

Matrix
<class 'numpy.ndarray'>
[[0.         0.         0.         ... 0.         0.03346155 0.        ]
 [0.02521404 0.02521404 0.02521404 ... 0.02521404 0.         0.05042808]]


### Doc word dataframe

In [56]:
abc=pd.DataFrame(rev_matrix_arr, columns=tfidf.get_feature_names())

In [57]:
abc.to_csv('norahray_check.csv')

### Cosine similarity (Based on Clean Review)
. This is what we can regenerate based on input parmeters (I think)

In [53]:
cosine_similarities = cosine_similarity(rev_matrix)

In [54]:
len(cosine_similarities)

2

In [55]:
similarities = {}

In [2114]:
for i in range(len(cosine_similarities)):
    
    # Sort each element in cosine_similarities and get the indexes of the artists
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    
    # Store in similarities each name of the 10 most similar artists
    # Keep the first one as the main search item - and then the remaining similar artists 
    similarities[df['album'].iloc[i]] = [(cosine_similarities[i][x], df['album'][x], df['artist'][x]) for x in similar_indices][0:]

**Enter an album name to get the similarity scores of teh album**

In [2960]:
print(similarities['Day Breaks'])
#print(similarities.values()[0].keys()[0])

[(1.0000000000000007, 'Day Breaks', 'Norah Jones'), (0.168611616858465, 'Swiss Radio Days Vol. 41 - Zurich 1961', 'Ray Charles'), (0.13242978170917313, 'ArtScience', 'Robert Glasper Experiment'), (0.10834982269422186, 'Harmony of Difference EP', 'Kamasi Washington'), (0.10700207410955759, 'Covered', 'Robert Glasper Trio'), (0.10571292049776597, 'Zamazu', 'Roberto Fonseca'), (0.10019935187703619, 'Emanon', 'Wayne Shorter'), (0.09658133203436345, 'Lady Day: The Master Takes and Singles', 'Billie Holiday'), (0.09573164299469586, 'Both Directions at Once: The Lost Album', 'John Coltrane'), (0.08467306551940389, 'First Time Out: Live at Slugs 1967 / Duo Exchange: Complete Sessions', 'Rashied Ali / Frank Lowe / Rashied Ali Quintet'), (0.079788992009909, 'Bitches Brew [Legacy Edition]', 'Miles Davis'), (0.07941305657884933, 'Dreams and Daggers', 'Cécile McLorin Salvant'), (0.07924444859869917, 'To Be Free: The Nina Simone Story', 'Nina Simone'), (0.07858338531747182, 'Winter in America', 'Gil

In [2]:
norah="The restive jazz star, who went on to work with Jack White, Danger Mouse, and others after¬†her multiplatinum debut, returns with a no-frills record that fades into the background without much fuss. At this point of her career, it‚Äôs easy to forget that Norah Jones was once considered a jazz singer. Back in 2002, long before she worked with Jack White and Brian ‚ÄúDanger Mouse‚Äù Burton, Jones released Come Away With Me, an understated gem that went on to sell over 11 million units and earn eight Grammy awards‚including Album of the Year and Song of the Year for ‚Don't Know Why. ‚Armed with that voice‚wry, simmering inflection‚the Texas native has proven she can sing anything, and sound natural doing so, no matter where the road has taken her. That said, its been tough to assess Jones creative arc. Feels Like Home, her 2004 sophomore LP, eschewed the traditional jazz of the first record for a folk/country sound. She's released two honky-tonk country albums as a member of the Little Willies; and in 2012, Jones dropped Little Broken Hearts, a concept record about emotional anguish, set against Danger Mouse‚ soulful pop arrangements. Elsewhere, Jones had notable features on Mouse and composer Daniele Luppi‚Rome; with Q-Tip, OutKast and Talib Kweli on their respective albums; and on the Robert Glasper Experiment  Black Radio 2. Throughout all this, Jones has remained somewhat anonymous, mainly because she‚Äôs never appealed to pop and hip-hop consumers. Her music speaks to the adult contemporary crowd, those who actually buy CDs instead of streaming online. Jones has maintained a nice career largely out of the public eye; now 14 years removed from her seminal work, the musician returns to initial form on Day Breaks, her sixth studio album. In a recent interview with the New York Times, Jones said the genesis of the LP can be traced back to a 2014 performance at the Kennedy Center in Washington, D.C., during a 75th anniversary concert for her label, Blue Note Records. She was performing a cover of songwriter Jesse Harris‚Äô ‚ÄúI‚Äôve Got to See You Again‚Äù with iconic saxophonist Wayne Shorter, drummer Brian Blade, and bassist John Patitucci. ‚ÄúWhen I started thinking about making a ‚Äòjazz record,‚Äô mostly I was thinking about recording with Wayne and Brian,‚Äù Jones told the Times. ‚ÄúI didn‚Äôt want it to be standards. I was hoping for something very rhythmic, with Wayne floating over the top.‚Äù Shorter, Blade, and organist Dr. Lonnie Smith appear throughout Day Breaks, an efficient 12-track set featuring nine original songs and three covers: Horace Silver‚Äôs ‚ÄúPeace,‚Äù Duke Ellington‚Äôs ‚ÄúFleurette Africaine‚Äù and Neil Young‚Äôs ‚ÄúDon‚Äôt Be Denied.‚Äù Day Breaks is especially sparse, a no-frills record that fades into the background without much fuss. It seems to reflect the singer‚Äôs personal and professional comfort, that‚Äîafter 15 years as a signed artist with more than 50 million records sold‚ÄîJones doesn‚Äôt need to adhere to industry pressures to remain relevant. Whereas some artists revert to their best-received work as a way to reignite past glory, Day Breaks feels like the logical next step for a singer who‚Äôs done just about everything there is to do musically. This one isn't a barn-burner, but it's not supposed to be. Unfortunately, though, Day Breaks grows a bit tedious near the middle, and it's easy to forget it's playing if you aren't paying attention. Lyrically, Day Breaks¬†embraces¬†a hazy‚Äîif not melancholy‚Äîvibe, similar to the Billie Holiday albums Jones listened to as a child. These tracks address some level of perseverance, a pushing through to better days whether romantically or socially. Jones uses a conversational cadence to tell these stories, bolstering the narrative and giving her words better impact. On ‚ÄúFlipside,‚Äù a song about racial and civic injustice, Jones and the band sound especially poignant. The other songs slow to a crawl, but this one is hard-charging with a strong message. ‚ÄúIf we‚Äôre all free, then why does it seem we can‚Äôt just be?‚Äù Jones asks. ‚ÄúPut the guns away or we‚Äôre all gonna lose.‚Äù (As of this review, two more unarmed black men‚ÄîTerence Crutcher in Tulsa, Oklahoma; Keith Lamont Scott in Charlotte, North Carolina‚Äîwere shot and killed by police for no reason.) And while Jones didn‚Äôt write the lyrics to ‚ÄúPeace‚Äù‚Äîit is a cover, after all‚Äîshe effectively owns the message. ‚Peace,she hums, is for everyone.‚Äù Sure that‚Äôs a noble idea, but w've got a long way to go."
charles="best new reissue At last available digitally, these quietly revolutionary albums still break one of music‚Äôs deepest-held genre orthodoxies. More than a half-century later, the premise of Ray Charles‚Äô two-volume Modern Sounds in Country and Western Music‚Äîthat is, an established R&B star makes a record of country hits‚Äîmight yet inspire a spit take from the uninitiated. Alas, the uninitiated have long remained uninitiated, since these groundbreaking albums have been reissued infrequently since their 1962 release. But a remastered Concord set, which includes vinyl and CD reprints of both albums, marks the first time these landmarks are available as downloads and streams. Maybe having them available outside of distorted YouTube rips will, 57 years later, affirm Charles‚Äô rightful status as a country music pioneer. Building off a string of hits with Atlantic and an ABC-Paramount contract with a ‚Äúfull artistic freedom‚clause, Charles was finally able to record the music he‚Äôd been listening to since he was a self-described country boy growing up in Greenville, FL., a town of around 900 near the Georgia border. Back then, he would tune into the Grand Ole Opry every Saturday night. That period provided direct inspiration for the first Modern Sounds and its hastily assembled sequel, recorded in the wake of its predecessor‚ explosive success. Charles put his golden touch on two dozen of the biggest country songs of the day. ‚ÄúI think a lot of the hillbilly music is wonderful,‚Äù he told Billboard, starting his campaign to record such tunes in 1959. ‚ÄúI think I could do a good job with the right hillbilly song today. If you really have the ability, that‚Äôs what counts.‚Äù Despite several subsequent counterexamples, from Charley Pride to Darius Rucker, the idea that a black singer could attain mainstream success singing country music still challenges one of the deepest-held genre orthodoxies. ‚ÄúA rhythm-and-blues singer, at least when I was coming up, was typecast, much like an actor,‚Äù Charles wrote in his 1978 autobiography. ‚ÄúEven today I‚Äôm called ‚ÄòRay Charles, the blues singer,‚Äô or ‚ÄòRay Charles, the soul singer.‚Äô And this is after 20 years of singing damn near everything!‚Äù Charles approached the album with what would turn out to be necessary defiance, as he faced skepticism from his label and the industry at large. Even though the albums contain some of Charles‚Äô most enduring singles, they are still more accepted as Ray Charles songs than disclaimer-free country. When Modern Sounds was first released, the charts refused to track this music as country; the Grammys nominated these very clearly named albums in rhythm & blues categories. Still, these arrangements played off the perceived tension of Charles‚Äô foray into Nashville territory. Half the songs were recorded in Los Angeles, with Charles backed by lush strings and a straight-laced choir to build a poppier take on the pervasive Nashville Sound. In New York, a swinging big band more familiar to those who were already Charles fans supported the other half. The end result is seamless, showing just how wide a gap Charles‚Äô voice could bridge. String-backed ballad ‚ÄúI Can‚Äôt Stop Loving You‚Äù is a larger-than-life song that toes the line of pure schlock with lyrical hyperbole and musical gloss. But a gentle, insistent country shuffle and Charles‚Äô endless capacity for relatable sorrow (perhaps country music‚Äôs most enduring hallmark) ground it. Buried in the album‚Äôs B-side because producer Sid Feller thought it was the weakest track, the song became the biggest hit of Charles‚Äô career. Charles‚Äô mix of polish and pure emotion has proven nearly impossible to match, from weepy orchestral tunes like ‚ÄúYou Don‚Äôt Know Me‚Äù (maybe the most convincing description of life in the friend zone ever recorded) to brassy up-tempo numbers, like Hank Williams‚Äô ‚ÄúHey, Good Lookin‚Äô.‚Äù But making something infallibly bright and genuine, rich and timeless was a secondary achievement. More importantly, he proved that the connection he heard between country‚Äôs twang and R&B‚Äôs groove wasn‚Äôt a fatal distinction but simply a matter of interpretation. Charles exposed how inane it is to draw a hard line between two genres that shared an ancestor‚Äîthe United States‚Äô original hybrid, the blues. You can go through and parse which parts of every record sound more like Nashville, which parts sound more like soul. ‚ÄúYou Are My Sunshine‚Äù relies on the same hip-swiveling call-and-response that Motown would soon make ubiquitous. ‚ÄúBorn to Lose‚Äù uses barely any swing outside of Charles‚Äô voice, readying it for the honky-tonk. But that kind of dissection misses the point‚Äîthat this music is accessible to him, too, a black man. In the end, Charles didn‚Äôt just fit in; he revolutionized the genre by sparking a rush of Nashville/pop crossover acts. This music remains a tribute to and rejoinder of the futile divisions we so often take for granted. Charles made headlines promoting the album with integrated shows throughout the South, refusing to perform in segregated venues. ‚ÄúBoth whites and Negroes alike have attended the same dance and actually danced together at Charles concerts,‚Äù wrote the Chicago Daily Defender in November 1962 of Charles‚Äô shows in Memphis and St. Louis. ‚ÄúThere is no race problem when he plays.‚Äù It‚Äôs proof of Charles‚Äô enduring yet still-aspirational achievement on these albums: unity"